In [ ]:
in_colab = False

# Project 13*: Machine Learning of Many Body Localization (Exact diagonalization + Machine Learning)

Use exact diagonalization to obtain all eigenstates of the the Heisenberg model with a
random field, 

\begin{equation}
    H = J \sum_i \vec{S}_{i} \cdot \vec{S}_{i+1} - \sum_i h_i S^z_i
\end{equation}

, where the values of the field $ h_i \in [-W, W] $ are chosen from a uniform random distribution with a "disorder strength" $W$ (Use moderate system sizes $L = [10, 12]$). 

The exciting property of this model is that it is believed to undergo a phase transition from an extended phase (small $W$) to a localized phase (large $W$). 

We will use ML to detect this transition: Pick a number of eigenstates that are near energy $E = 0$ and obtain the reduced density matrices $\rho^A$, where $A$ is a region of $n$ consecutive spins (a few hundred to thousands eigenstates for different disorder realizations). 

Now use the density matrices for $W = 0.5 J$ and $W = 8.0 J$ to train a neural network (just interpret the entries of $\rho^A$ as an image with $2^n \times 2^n$ pixel). Then use this network and study the output of the neural network for different $W$. 

How does the results depend on system size $L$ and block size $n$? At which $W_c$ do you expect the
transition to occur?

_Author: Tin Kei CHENG_  
_TUM, 2021_

## Literature

Wikipedia:  
[Many body localization](https://en.wikipedia.org/wiki/Many_body_localization)  
[Localization protected quantum order](https://en.wikipedia.org/wiki/Localization_protected_quantum_order)  

Many-body localization (MBL) is a dynamical phenomenon which leads to the breakdown of equilibrium statistical mechanics in isolated many-body systems. Such systems never reach local thermal equilibrium, and retain local memory of their initial conditions for infinite times.

MBL was first proposed by P.W. Anderson in 1958 as a possibility that could arise in strongly disordered quantum systems. The basic idea was that if particles all live in a random energy landscape, then any rearrangement of particles would change the energy of the system. Since energy is a conserved quantity in quantum mechanics, such a process can only be virtual and cannot lead to any transport of particle number or energy.  

The process of thermalization erases local memory of the initial conditions. In textbooks, thermalization is ensured by coupling the system to an external environment or "reservoir," with which the system can exchange energy. What happens if the system is isolated from the environment, and evolves according to its own Schrödinger equation? Does the system still thermalize?

Quantum mechanical time evolution is unitary and formally preserves all information about the initial condition in the quantum state at all times.

This question can be formalized by considering the quantum mechanical density matrix ρ of the system. If the system is divided into a subregion A (the region being probed) and its complement B (everything else), then all information that can be extracted by measurements made on A alone is encoded in the reduced density matrix $\rho_A = Tr_B (\rho(t))$. If in the long time limit $\rho_A(t)$ approaches a thermal density matrix at a temperature set by the energy density in the state, then the system has "thermalized," and no local information about the initial condition can be extracted from local measurements. This process of "quantum thermalization" may be understood in terms of B acting as a reservoir for A. In this perspective, the entanglement entropy $ S = - Tr \rho_A log \rho_A $ of a thermalizing system in a pure state plays the role of thermal entropy. Thermalizing systems therefore generically have extensive or "volume law" entanglement entropy at any non-zero temperature.

In contrast, if $\rho_A(t)$ fails to approach a thermal density matrix even in the long time limit, and remains instead close to its initial condition $\rho_A(0)$, then the system retains forever a memory of its initial condition in local observables. This latter possibility is referred to as "many body localization," and involves B failing to act as a reservoir for A. Eigenstates of systems exhibiting MBL do not obey the ETH, and generically follow an "area law" for entanglement entropy (i.e. the entanglement entropy scales with the surface area of subregion A).

In thermalizing systems, energy eigenstates have volume law entanglement entropy. In MBL systems, energy eigenstates have area law entanglement entropy.

In thermalizing systems, entanglement entropy grows as a power law in time starting from low entanglement initial conditions. In MBL systems, entanglement entropy grows logarithmically in time starting from low entanglement initial conditions.

In thermalizing systems, the dynamics of out-of-time-ordered correlators forms a linear light cone which reflects the ballistic propagation of information. In MBL systems, the light cone is logarithmic.

What's more, while individual eigenstates aren't themselves experimentally accessible, order in eigenstates nevertheless has measurable dynamical signatures. The eigenspectrum properties change in a singular fashion as the system transitions between from one type of MBL phase to another, or from an MBL phase to a thermal one---again with measurable dynamical signatures.

## Imports

In [ ]:
import os
import sys
import copy
import json
import gzip
import lzma
import pytz
import time
import pickle
import numpy as np
from numba import jit, njit # Set "nopython" mode for best performance, equivalent to @njit # cache=True, parallel=True
from datetime import datetime
from tqdm import tqdm

tz = pytz.timezone('Europe/Berlin')

import scipy
import scipy.linalg
import scipy.sparse.linalg

from scipy.sparse import csr_matrix, kron, identity
from scipy.sparse.linalg import eigsh
from scipy.linalg import svd
from scipy.optimize import curve_fit

from collections import OrderedDict

%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator

dpi = 100
fig_w = 1280
fig_h = 640

%matplotlib inline

In [ ]:
if in_colab:
    ED_data_dir = 'drive/MyDrive/Colab Data/CMMP/ED_data'
    rho_train_data_dir = 'drive/MyDrive/Colab Data/CMMP/rho_train_data'
    rho_valid_data_dir = 'drive/MyDrive/Colab Data/CMMP/rho_valid_data'
    model_dir  = 'drive/MyDrive/Colab Data/CMMP/models'
    signal_dir = 'drive/MyDrive/Colab Data/CMMP'
else:
    ED_data_dir = 'ED_data'
    rho_train_data_dir = 'rho_train_data'
    rho_valid_data_dir = 'rho_valid_data'
    model_dir  = 'models'
    signal_dir = '.'

In [ ]:
if in_colab:
    !cat /proc/cpuinfo

In [ ]:
if in_colab:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
# if in_colab:
#     !pip install pytorch_lightning==0.7.6 torchsummary==1.5.1
#     !pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# pip install torch==1.4.0+cu92 torchsummary==1.5.1 torchvision==0.5.0+cu92 pytorch-lightning==0.7.6  -f https://download.pytorch.org/whl/torch_stable.html
# import torch
# from torch.utils.data import DataLoader
# from torchvision import transforms

# import pytorch_lightning as pl
# from pytorch_lightning import Trainer, seed_everything
# from pytorch_lightning.callbacks import EarlyStopping
# from pytorch_lightning.loggers import TensorBoardLogger
# # from torchsummary import summary
# # help(summary)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = "cpu"
# print(device)
# # python -c "import torch; print(torch.__version__)"

## Util functions

In [ ]:
def dt():
    return datetime.now(tz=tz).strftime('%Y-%m-%d %H:%M:%S')

def dict_to_str(_dict):
    
    od = OrderedDict(sorted(_dict.items())) # Sort keys.
    s = json.dumps(od) # Turn dict into str.
    s = s[1:-1].replace('\"', '').replace(' ', '') # Replace some special characeters.
    s = ''.join(x if x.isalnum() else ('=' if x == ':' else '_') for x in s) # Replace all remaining special characters.

    return s


def save_cache(obj, obj_name, obj_params, cache_dir='cache'):
    """Cache an object, together with the parameters used to generate it.
    For `obj_params`, try not to use nested dict or with complicated objects.

    Parameters
    ----------
    obj : object
        An `object` you want to cache.
    obj_name : str
        A unique name you give to this object.
    obj_params : dict
        A `dict` of all parameters necessary to generate this object.
    cache_dir : str, optional
        Directory where the cache is located.
    """

    param_str = dict_to_str(obj_params)
    os.makedirs(os.path.join(cache_dir, obj_name), exist_ok=True)
    with gzip.open(os.path.join(cache_dir, obj_name, param_str + '.pkl.gz'), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


def load_cache(obj_name, obj_params, cache_dir='cache'):
    """Check if the object is cached. If not, return None.
    For `obj_params`, try not to use nested dict or with complicated objects.

    Parameters
    ----------
    obj_name : str
        A unique name you give to this object.
    obj_params : dict
        A `dict` of all parameters necessary to generate this object.
    cache_dir : str, optional
        Directory where the cache is located.
    """

    param_str = dict_to_str(obj_params)
    os.makedirs(os.path.join(cache_dir, obj_name), exist_ok=True)
    if os.path.isfile(os.path.join(cache_dir, obj_name, param_str + '.pkl.gz')):
        with gzip.open(os.path.join(cache_dir, obj_name, param_str + '.pkl.gz'), 'rb') as handle:
            obj = pickle.load(handle)
            return obj
    else:
        return None


def check_shutdown_signal(signal_dir=signal_dir):
    """To gracefully stop generating data by making sure a loop is completed, this function will read a text file in a directory for the value `1`.
    
    Return
    ------
    shutdown : bool
        Shutdown signal detected.
    """

    os.makedirs(os.path.join(signal_dir), exist_ok=True)
    if os.path.isfile(os.path.join(signal_dir, 'shutdown_signal.txt')):
        with open(os.path.join(signal_dir, 'shutdown_signal.txt')) as f:
            lines = f.readlines()
        if lines is not None and len(lines) > 0:
            lines = [x.strip() for x in lines]
            if lines[0] == '1':
                return True

    return False


@njit
def is_sorted(arr):
    return np.all(arr[:-1] <= arr[1:])


def save_ED(obj, obj_name, L, W, periodic, data_dir=ED_data_dir):
    """Save a list of exact diagonalization results, organized by the parameters used to generate them.
    For `obj_params`, try not to use nested dict or with complicated objects.

    Parameters
    ----------
    obj : list
        A list of lists, where each reduced density matrix is paired with its disorder strength W.
        Number of data must be a multiple of 10.
    obj_name : str
        A unique name you give to this object. Call it `rho_A`.
    L : int
        System size.
    W : float
        Disorder strength.
    periodic : bool
        Whether the Hamiltonian is periodic.
    data_dir : str, optional
        Directory where the data is saved.
    """

    directory = os.path.join(data_dir, obj_name, 'L={:02d}'.format(L), 'W={:.2f}'.format(W), 'periodic={}'.format(periodic))
    os.makedirs(directory, exist_ok=True)

    # Check if file exists, and increment suffix.
    i = 0
    while os.path.exists(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i))):
        i += 1

    with gzip.open(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i)), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


def load_ED(obj_name, L, W, periodic, data_dir=ED_data_dir):
    """Check if the object is cached. If not, return None.
    For `obj_params`, try not to use nested dict or with complicated objects.

    Parameters
    ----------
    obj_name : str
        A unique name you give to this object. Call it `rho_A`.
    L : int
        System size.
    W : float
        Disorder strength.
    periodic : bool
        Whether the Hamiltonian is periodic.
    data_dir : str, optional
        Directory where the data is saved.
    """

    raise NotImplementedError('Function not implemented.')

    param_str = dict_to_str(obj_params)
    os.makedirs(os.path.join(data_dir, obj_name, 'L={:2d}'.format(L), 'W={:.2d}'.format(W)), exist_ok=True)
    if os.path.isfile(os.path.join(data_dir, obj_name, 'L={:2d}'.format(L), 'W={:.2d}'.format(W), param_str + '.pkl.gz')):
        with gzip.open(os.path.join(data_dir, obj_name, 'L={:2d}'.format(L), 'W={:.2d}'.format(W), param_str + '.pkl.gz'), 'rb') as handle:
            obj = pickle.load(handle)
            return obj
    else:
        return None


In [ ]:
def save_rho_train(obj, obj_name, L, n, periodic, num_EV, data_dir=rho_train_data_dir):
    """Save a list of reduced density matrices with W = {0.5, 8}.

    Parameters
    ----------
    obj : list
        A list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. obj[i][0] is a 2D numpy.ndarray of the reduced density matrix, and obj[i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    obj_name : str
        A name you give to this object. Call it `rho_A`.
    L : int
        System size.
    n : int
        Number of consecutive spins sampled.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_EV : int
        Number of eigenvalues around zero being sampled.
    data_dir : str, optional
        Directory where the data is saved.
    """

    directory = os.path.join(data_dir, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    # Check if file exists, and increment suffix.
    i = 0
    while os.path.exists(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i))):
        i += 1

    with gzip.open(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i)), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


def save_rho_valid(obj, obj_name, L, n, periodic, num_EV, data_dir=rho_valid_data_dir):
    """Save a list of reduced density matrices with random W != {0.5, 8}.

    Parameters
    ----------
    obj : list
        A list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. obj[i][0] is a 2D numpy.ndarray of the reduced density matrix, and obj[i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    obj_name : str
        A name you give to this object. Call it `rho_A`.
    L : int
        System size.
    n : int
        Number of consecutive spins sampled.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_EV : int
        Number of eigenvalues around zero being sampled.
    data_dir : str, optional
        Directory where the data is saved.
    """

    directory = os.path.join(data_dir, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    # Check if file exists, and increment suffix.
    i = 0
    while os.path.exists(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i))):
        i += 1

    with gzip.open(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i)), 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
def load_rho_train(obj_name, L, n, periodic, num_EV, data_dir=rho_train_data_dir):
    """Load a list of reduced density matrices with W = {0.5, 8}.

    Parameters
    ----------
    obj_name : str
        A name you give to this object. Call it `rho_A`.
    L : int
        System size.
    n : int
        Number of consecutive spins sampled.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_EV : int
        Number of eigenvalues around zero being sampled.
    data_dir : str, optional
        Directory where the data is saved.

    Return
    ------
    obj : list
        A list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. obj[i][0] is a 2D numpy.ndarray of the reduced density matrix, and obj[i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    """

    directory = os.path.join(data_dir, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    # Check if file exists, load the file, and increment suffix.
    i = 0
    data = []
    while os.path.exists(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i))):
        with gzip.open(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i)), 'rb') as handle:
            data = data + pickle.load(handle)
        i += 1

    return data


def load_rho_valid(obj_name, L, n, periodic, num_EV, data_dir=rho_valid_data_dir):
    """Load a list of reduced density matrices with random W != {0.5, 8}.

    Parameters
    ----------
    obj_name : str
        A name you give to this object. Call it `rho_A`.
    L : int
        System size.
    n : int
        Number of consecutive spins sampled.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_EV : int
        Number of eigenvalues around zero being sampled.
    data_dir : str, optional
        Directory where the data is saved.

    Return
    ------
    obj : list
        A list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. obj[i][0] is a 2D numpy.ndarray of the reduced density matrix, and obj[i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    """

    directory = os.path.join(data_dir, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    # Check if file exists, load the file, and increment suffix.
    i = 0
    data = []
    while os.path.exists(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i))):
        with gzip.open(os.path.join(directory, obj_name + '-{:09d}.pkl.gz'.format(i)), 'rb') as handle:
            data = data + pickle.load(handle)
        i += 1

    return data


In [ ]:
def save_model(model, file_name, L, n, periodic, num_EV, directory=model_dir):
    """Save model as pickle"""

    model = model.cpu()
    model_dict = {
        "state_dict": model.state_dict(),
        "hparams": model.hparams
    }

    directory = os.path.join(directory, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    model_path = os.path.join(directory, file_name)
    with gzip.open(model_path, 'wb', 4) as handle:
        pickle.dump(model_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return model_path


def load_model(file_name, L, n, periodic, num_EV, directory=model_dir):

    directory = os.path.join(directory, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    model_path = os.path.join(directory, file_name)
    with gzip.open(model_path, 'rb') as fp:
        return pickle.load(fp)["state_dict"]


def model_exists(file_name, L, n, periodic, num_EV, directory=model_dir):

    directory = os.path.join(directory, 'L={:02d}'.format(L), 'n={:02d}'.format(n), 'periodic={}'.format(periodic), 'num_EV={}'.format(num_EV))
    os.makedirs(directory, exist_ok=True)

    model_path = os.path.join(directory, file_name)
    return os.path.exists(model_path)


## Build Hamiltonian and Exact Diagonalization

### Functions

In [ ]:
@njit
def get_h(L, W):
    h = np.random.uniform(-W, W, L)
    return h


In [ ]:
def build_si_list(L):

    # Get single site operaors.
    sx = csr_matrix(np.array([[0.,  1. ], [1. ,  0.]]))
    sy = csr_matrix(np.array([[0., -1.j], [1.j,  0.]]))
    sz = csr_matrix(np.array([[1.,  0. ], [0. , -1.]]))
    id = csr_matrix(np.eye(2))

    # ========================================
    # Start cached area: si_list.
    # ========================================

    obj_params = {'L': L}

    sx_list = load_cache('sx_list', obj_params)
    sy_list = load_cache('sy_list', obj_params)
    sz_list = load_cache('sz_list', obj_params)

    if sx_list is None or sy_list is None or sz_list is None:

        # print('Cache not found for `si_list`. Generate from scratch.')

        sx_list = []  # sx_list[i] = kron([id, id, ..., id, sx, id, .... id])
        sy_list = []
        sz_list = []

        for i_site in range(L):

            x_ops = [id] * L
            y_ops = [id] * L
            z_ops = [id] * L
            x_ops[i_site] = sx
            y_ops[i_site] = sy
            z_ops[i_site] = sz

            X = x_ops[0]
            Y = y_ops[0]
            Z = z_ops[0]
            for j in range(1, L):
                X = kron(X, x_ops[j], 'csr')
                Y = kron(Y, y_ops[j], 'csr')
                Z = kron(Z, z_ops[j], 'csr')
            sx_list.append(X)
            sy_list.append(Y)
            sz_list.append(Z)

        save_cache(sx_list, 'sx_list', obj_params)
        save_cache(sy_list, 'sy_list', obj_params)
        save_cache(sz_list, 'sz_list', obj_params)

    # else:

    #     print('Cache found for `si_list`. Load from cache.')

    # ========================================
    # End cached area: si_list.
    # ========================================

    return sx_list, sy_list, sz_list


In [ ]:
def build_H_ii(L, periodic):

    sx_list, sy_list, sz_list = build_si_list(L)

    # ========================================
    # Start cached area: H_ii.
    # ========================================
    
    obj_params = {'L': L, 'periodic': periodic}

    H_xx = load_cache('H_xx', obj_params)
    H_yy = load_cache('H_yy', obj_params)
    H_zz = load_cache('H_zz', obj_params)

    if H_xx is None or H_yy is None or H_zz is None:

        # print('Cache not found for `H_ii`. Generate from scratch.')

        H_xx = csr_matrix((2**L, 2**L))
        H_yy = csr_matrix((2**L, 2**L))
        H_zz = csr_matrix((2**L, 2**L))

        for i in range(L if periodic else L - 1):
            H_xx = H_xx + sx_list[i] * sx_list[(i + 1) % L]
            H_yy = H_yy + sy_list[i] * sy_list[(i + 1) % L]
            H_zz = H_zz + sz_list[i] * sz_list[(i + 1) % L]

        save_cache(H_xx, 'H_xx', obj_params)
        save_cache(H_yy, 'H_yy', obj_params)
        save_cache(H_zz, 'H_zz', obj_params)

    # else:

    #     print('Cache found for `H_ii`. Load from cache.')

    # ========================================
    # End cached area: H_ii.
    # ========================================

    return H_xx, H_yy, H_zz, sx_list, sy_list, sz_list


In [ ]:
def build_H(L, W, J, periodic=False):

    H_xx, H_yy, H_zz, sx_list, sy_list, sz_list = build_H_ii(L, periodic)

    # H_z is not cached due to randomness.
    H_z  = csr_matrix((2**L, 2**L))
    h    = get_h(L, W)

    for i in range(L):
        H_z = H_z + h[i] * sz_list[i]

    H = J * (H_xx + H_yy + H_zz) - H_z

    return H

def build_Hs(L, W, J, periodic=False, num_Hs=1000):

    H_xx, H_yy, H_zz, sx_list, sy_list, sz_list = build_H_ii(L, periodic)

    Hs = []
    for i in tqdm(range(num_Hs), leave=False, desc='build_Hs()'):

        # H_z is not cached due to randomness.
        H_z  = csr_matrix((2**L, 2**L))
        h    = get_h(L, W)

        for i in range(L):
            H_z = H_z + h[i] * sz_list[i]

        H = J * (H_xx + H_yy + H_zz) - H_z
        Hs.append(H)

    return Hs


In [ ]:
@njit
def ED(H):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.

    The column V[:, i] is the normalized eigenvector corresponding to the eigenvalue E[i].
    Will return a matrix object if a is a matrix object.

    Parameters
    ----------
    H : numpy.ndarray
        Hamiltonian to diagonalize.

    Return
    ------
    E : 1D numpy.ndarray
        Eigenvalues, sorted in ascending order.
    V : 2D numpy.ndarray
        Eigenvectors.
    """

    # if L >= 20:
    #     warnings.warn("Large L: Exact diagonalization might take a long time!")

    E, V = np.linalg.eigh(H)

    assert is_sorted(E), 'Eigenvalues not sorted!'

    return E, V


def ED_sparse(H, k):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.

    An array representing the k eigenvectors. The column v[:, i] is the eigenvector corresponding to the eigenvalue w[i].

    Parameters
    ----------
    H : numpy.ndarray
        Hamiltonian to diagonalize.
    k : int
        Number of eigenvalues around E = 0 to obtain.

    Return
    ------
    E : 1D numpy.ndarray
        Eigenvalues, sorted in ascending order.
    V : 2D numpy.ndarray
        Eigenvectors.
    """

    # if L >= 20:
    #     warnings.warn("Large L: Exact diagonalization might take a long time!")

    E, V = scipy.sparse.linalg.eigsh(H, k=k, sigma=0, which='LM', return_eigenvectors=True)
    sorted_indices = np.abs(E).argsort()
    E = E[sorted_indices]
    V = V[:, sorted_indices]

    assert is_sorted(np.abs(E)), 'Eigenvalues not sorted!'

    return E, V


def EDs(Hs):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.

    The column V[:, i] is the normalized eigenvector corresponding to the eigenvalue E[i].
    Will return a matrix object if a is a matrix object.

    Parameters
    ----------
    Hs : list of scipy.sparse.csr_matrix
        A list of Hamiltonians to diagonalize.

    Return
    ------
    E : list of 1D numpy.ndarray
        Eigenvalues of each Hamiltonian, sorted in ascending order.
    V : list of 2D numpy.ndarray
        Eigenvectors of each Hamiltonian.
    """

    # if L >= 20:
    #     warnings.warn("Large L: Exact diagonalization might take a long time!")

    Es = []
    Vs = []
    for H in Hs:

        # Can't use scipy's eigsh, because we need ALL eigenwhatevers.
        # E, V = eigsh(H, k=10, which='SM', return_eigenvectors=True)
        # E, V = np.linalg.eigh(H.A)
        E, V = ED(H.toarray())
        Es.append(E)
        Vs.append(V)

    return Es, Vs


def EDs_sparse(Hs, k):
    """For comparison: obtain ground state energy from exact diagonalization.

    Exponentially expensive in L, only works for small enough `L` <~ 20.

    An array representing the k eigenvectors. The column v[:, i] is the eigenvector corresponding to the eigenvalue w[i].

    Parameters
    ----------
    Hs : list of scipy.sparse.csr_matrix
        A list of Hamiltonians to diagonalize.
    k : int
        Number of eigenvalues around E = 0 to obtain.

    Return
    ------
    E : 1D numpy.ndarray
        Eigenvalues, sorted in ascending order.
    V : 2D numpy.ndarray
        Eigenvectors.
    """

    # if L >= 20:
    #     warnings.warn("Large L: Exact diagonalization might take a long time!")

    Es = []
    Vs = []
    for H in tqdm(Hs, leave=False, desc='EDs_sparse()'):

        E, V = scipy.sparse.linalg.eigsh(H, k=k, sigma=0, which='LM', return_eigenvectors=True)
        sorted_indices = np.abs(E).argsort()
        E = E[sorted_indices]
        V = V[:, sorted_indices]
        Es.append(E)
        Vs.append(V)

    return Es, Vs



In [ ]:
# @njit
def select_N_eigenvalues(E, V, n, where='zeroest'):
    """
    Select N eigenvalues closest to the lowest, to zero, or to the highest.

    Parameters
    ----------
    E : 1D numpy.ndarray
        Sorted eigenvalues in ascending order.
    V : 2D numpy.ndarray
        Corresponding eigenvectors.
        The column V[:, i] is the normalized eigenvector corresponding to the eigenvalue E[i].
    n : int
        Number of eigenvalues to store.
    where : str
        Where to select the eigenvalues. where = {'lowest', 'zeroest', 'highest'}
    """

    if where == 'lowest':
        E = E[:n]
        V = V[:, :n]
    elif where == 'highest':
        E = E[-n:]
        V = V[:, -n:]
    elif where == 'zeroest':
        # closest_indices = np.abs(E).argsort()[:n]
        # E = E[closest_indices]
        # V = V[:, closest_indices]
        # Faster implementation, but Numba doesn't support this. Still, it is a few microseconds faster.
        # Source: https://stackoverflow.com/questions/16817948/i-have-need-the-n-minimum-index-values-in-a-numpy-array
        closest_indices = np.argpartition(np.abs(E), n)[:n]
        E_temp = E[closest_indices]
        V_temp = V[:, closest_indices]
        sorted_indices = np.abs(E_temp).argsort()[:n]
        E = E_temp[sorted_indices]
        V = V_temp[:, sorted_indices]
        # assert np.all(E1 == E), 'Both sorting methods should be identical.'
        # assert np.all(V1 == V), 'Both sorting methods should be identical.'

    return E, V


### Computations

In [ ]:
# Test solving a single Hamiltonian.
L = 8
W = 0.5
J = 1
periodic = False
num_Hs = 10

H = build_H(L, W, J, periodic=False)
E, V = ED(H.toarray())
print('2^L: {}'.format(2**L))
print('#eigenvalues: {}'.format(len(E)))
print('E.shape: {}'.format(E.shape))
print('V.shape: {}'.format(V.shape))

In [ ]:
%%timeit
# Solve using numpy's dense solver
E, V = ED(H.toarray())
E0, V0 = select_N_eigenvalues(E, V, 20)

In [ ]:
%%timeit
# Solve using scipy's sparse solver instead.
E1, V1 = ED_sparse(H, 20)

In [ ]:
# Solve using numpy's dense solver
E, V = ED(H.toarray())
E0, V0 = select_N_eigenvalues(E, V, 20)
# Solve using scipy's sparse solver instead.
E1, V1 = ED_sparse(H, 20)

In [ ]:
# Check eigenvalues are sorted correctly, and eigenvectors are selected along the correct axis.
V0_norm = []
V1_norm = []
for i in range(len(E0)):
    V0_norm.append(np.linalg.norm(V0[:,i]))
    V1_norm.append(np.linalg.norm(V1[:,i]))

for E0i, E1i, V0i, V1i in zip(E0, E1, V0_norm, V1_norm):
    print('Numpy: {:+.12f} | Numpy norm: {:.16f}'.format(E0i, V0i))
    print('Scipy: {:+.12f} | Scipy norm: {:.16f}'.format(E1i, V1i))

assert np.allclose(V0i, V1i), 'Eigenvectors evaluated using Numpy and Scipy should be identical.'

In [ ]:
# Test solving multiple Hamiltonians.
Hs = build_Hs(L, W, J, periodic, num_Hs)
Es, Vs = EDs(Hs)
E0s, V0s = EDs_sparse(Hs, 20)

In [ ]:
def batch_generate_ED_data(L, W, J=1, periodic=False, num_Hs=1000, num_EV=20, save_data=False, npsp='sp'):

    obj_params = {'J': J, 'periodic': periodic} # Drop L and W. We use them for subdirectories.
    Hs = build_Hs(L, W, J, periodic, num_Hs)

    if npsp == 'sp':
        E0s, V0s = EDs_sparse(Hs, num_EV)
    else:
        Es, Vs = EDs(Hs)
        E0s = []
        V0s = []
        for E, V in zip(Es, Vs):
            E0, V0 = select_N_eigenvalues(E, V, num_EV)
            E0s.append(E0)
            V0s.append(V0)

    if save_data:
        save_ED(E0s, 'E0s', L, W, periodic)
        save_ED(V0s, 'V0s', L, W, periodic)

    return E0s, V0s


In [ ]:
# Test execution time.
J  = 1                      # Always = 1
Ws = [8]                    # Disorder strength W.
Ls = list(range(8,12))      # System size L.
ns = [1]*len(Ls)            # Number of samples for each L.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 20                 # Number of eigenvalues near zero to save.

for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            batch_generate_ED_data(L, W, J, p, num_Hs, num_EV, save_data=False, npsp='np')
            exec_time = time.time() - start_time
            et.append(exec_time)
            print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(fig_w/dpi,fig_h/dpi), dpi=dpi, squeeze=False)

base = 10
axes[0,0].plot(Ls, et)
axes[0,1].plot(np.array(Ls), np.log(et) / np.log(base))

axes[0,0].set_title('Execution time vs System size')
axes[0,1].set_title('Log(Execution time) vs System size')
axes[0,0].set_ylabel('Execution time')
axes[0,1].set_ylabel('Log(Execution time)')

for axe in axes:
    for ax in axe:
        ax.set_xlabel('System size L')
        # ax.legend(loc='best')
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
# Test execution time.
J  = 1                      # Always = 1
Ws = [8]                    # Disorder strength W.
Ls = list(range(8,16))      # System size L.
ns = [1]*len(Ls)            # Number of samples for each L.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 20                 # Number of eigenvalues near zero to save.

for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            batch_generate_ED_data(L, W, J, p, num_Hs, num_EV, save_data=False, npsp='sp')
            exec_time = time.time() - start_time
            et.append(exec_time)
            print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(fig_w/dpi,fig_h/dpi), dpi=dpi, squeeze=False)

base = 10
axes[0,0].plot(Ls, et)
axes[0,1].plot(np.array(Ls), np.log(et) / np.log(base))

axes[0,0].set_title('Execution time vs System size')
axes[0,1].set_title('Log(Execution time) vs System size')
axes[0,0].set_ylabel('Execution time')
axes[0,1].set_ylabel('Log(Execution time)')

for axe in axes:
    for ax in axe:
        ax.set_xlabel('System size L')
        # ax.legend(loc='best')
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
# Sample parameters.
J  = 1                                 # Always = 1
Ws = [0.5, 8] * 10 + [i/2 for i in range(1*2, 10*2)] # Disorder strength W.
Ls = [   8,   9,  10,  11, 12, 13, 14] # System size L.
ns = [1000, 500, 250, 100, 50, 20, 10] # Number of samples for each L.
ps = [False, True]                     # Periodic or not.

In [ ]:
# Test file size. It's not feasible to store all eigenvectors.
J  = 1             # Always = 1
Ws = [0.5, 8] * 10 + [i/2 for i in range(1*2, 10*2)] # Disorder strength W.
Ls = [   8]        # System size L.
ns = [1000]        # Number of samples for each L.
ps = [False, True] # Periodic or not.

fs = 0
for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            # batch_generate_ED_data(L, W, J, p, num_Hs)
            fs += 1
            exec_time = time.time() - start_time
            # print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))
print('Eigenvectors `Vs` dominates the file size. Assume 1000 samples are generated for each W, for around 20 Ws:')
print('For L=8, each Es file is 2MB, Hs is 23MB, Vs is 1000MB.')
print('Estimate file size of each run is thus {}MB'.format(fs*(2+23+1000)))

In [ ]:
print('The full eigenvectors `Vs` dominates the file size.')
for i in range(5):
    print('For L = {:2d}, Vs is {:3d} MB.'.format(8+i, 4**i))

## Extract reduced density matrix

### Summary

According to my limited understanding, a so-called partial trace is simply summing contributions of coefficients outside a chosen subsystem. 

Let's start with a simple example of two spins, divided into two subsystems $A$ and $B$, each with one spin:

\begin{equation}
    | s_1 \rangle \otimes | s_2 \rangle = | s_1 \rangle_A \otimes | s_2 \rangle_B = | A \rangle \otimes | B \rangle
\end{equation}

An eigenvector has the following basis:

\begin{equation}
    \begin{pmatrix}
        | (\downarrow)_A (\downarrow)_B \rangle \\
        | (\downarrow)_A (\uparrow)_B   \rangle \\
        | (\uparrow)_A   (\downarrow)_B \rangle \\
        | (\uparrow)_A   (\uparrow)_B   \rangle \\
    \end{pmatrix}
\end{equation}

Now add coefficients $C_i$ of the eigenvector:

\begin{equation}
    \begin{pmatrix}
        C_0 | (\downarrow)_A (\downarrow)_B \rangle \\
        C_1 | (\downarrow)_A (\uparrow)_B   \rangle \\
        C_2 | (\uparrow)_A   (\downarrow)_B \rangle \\
        C_3 | (\uparrow)_A   (\uparrow)_B   \rangle \\
    \end{pmatrix}
\end{equation}

Or $C_{i,j}$:

\begin{equation}
    \begin{pmatrix}
        C_{0,0} | (\downarrow)_A (\downarrow)_B \rangle \\
        C_{0,1} | (\downarrow)_A (\uparrow)_B   \rangle \\
        C_{1,0} | (\uparrow)_A   (\downarrow)_B \rangle \\
        C_{1,1} | (\uparrow)_A   (\uparrow)_B   \rangle \\
    \end{pmatrix}
\end{equation}

The reduce density matrix of subsystem $A$ is ${\rho}_A$ is defined as a partial trace over $B$:

\begin{equation}
    \rho_A = Tr_B(\rho)
\end{equation}

That can be obtained simply by summing coefficients related to $B$, thereby removing its contribution:

\begin{equation}
    \begin{aligned}[c]
        \begin{pmatrix}
            (C_0 + C_1) | (\downarrow)_A \rangle \\
            (C_2 + C_3) | (\uparrow)_A   \rangle \\
        \end{pmatrix}
    \end{aligned}
    \quad or \quad
    \begin{aligned}[c]
        \begin{pmatrix}
            (C_{0,0} + C_{0,1}) | (\downarrow)_A \rangle \\
            (C_{1,0} + C_{1,1}) | (\uparrow)_A   \rangle \\
        \end{pmatrix}
    \end{aligned}
\end{equation}

We observe that if we have an eigenvector in an array, the notation $C_i$ used on the left hand side is the array index, while the right hand side is the binary representation of said index. Does this relation extend beyond two spins? (Probably not, unless each spin is its own subsystem...)

But this is clearly wrong, because this is just a vector, not a matrix. Recall from Chapter 5 of our lecture notes that the definition of a reduced density matrix is also:

\begin{equation}
    \rho_A = Tr_B(\rho) = Tr_B(| \psi \rangle \langle \psi |) = \sum_{i',j,i,j} \psi^\ast_{i',j} \psi_{i,j} | i' \rangle \langle i | ,
\end{equation}

where $i$ are spins/sites related to subsystem $A$, and $j$ to $B$. An element of this matrix is simply:

\begin{equation}
    {(\rho_A)}_{i',i} = Tr_B(\rho)_{i',i} = \sum_{j} \psi^\ast_{i',j} \psi_{i,j} | i' \rangle \langle i |
\end{equation}

The basis $ | i' \rangle \langle i | $ is always implicit in all calculations and never really affect anything, ever, really. The only purpose is to make everything more complicated and confusing.

Reverting back to the notation of our previous 2-spin example, with $\psi_{i,j} \to C_i$ and $| i \rangle \to | s_1 \rangle_A$, a reduced density matrix can be constructed as follows:

\begin{equation}
    \begin{pmatrix}
        (C_0 + C_1)^\ast \times (C_0 + C_1) &                     ?               \\
                            ?               & (C_2 + C_3)^\ast \times (C_2 + C_3) \\
    \end{pmatrix}
\end{equation}

Now what should the off-diagonal terms look like? We have two choices:

\begin{equation}
    \begin{aligned}[c]
        \begin{pmatrix}
            (C_0 + C_1)^\ast \times (C_0 + C_1) & (C_2 + C_3)^\ast \times (C_0 + C_1) \\
            (C_0 + C_1)^\ast \times (C_2 + C_3) & (C_2 + C_3)^\ast \times (C_2 + C_3) \\
        \end{pmatrix}
    \end{aligned}
    \quad or \quad
    \begin{aligned}[c]
        \begin{pmatrix}
            (C_0 + C_1)^\ast \times (C_0 + C_1) & (C_0 + C_1)^\ast \times (C_2 + C_3) \\
            (C_2 + C_3)^\ast \times (C_0 + C_1) & (C_2 + C_3)^\ast \times (C_2 + C_3) \\
        \end{pmatrix}
    \end{aligned}
    \quad ...? 
\end{equation}

We are using a neural network and treating this as an image, so it would not matter if we flip the off-diagonal terms. The latter appears to be correct though, because:

\begin{equation}
    \begin{pmatrix}
        a^\ast_0 \\
        a^\ast_1
    \end{pmatrix}
    \otimes
    \begin{pmatrix}
        b_0 & b_1
    \end{pmatrix}
    \; = \;
    \begin{pmatrix}
        a^\ast_0 b_0 & a^\ast_0 b_1 \\
        a^\ast_1 b_0 & a^\ast_1 b_1
    \end{pmatrix}
\end{equation}

Finally, how to efficiently identify which coefficients such as $(C_0 + C_1)$ to sum, in a vectorized manner? All this nonsense is probably a single line in programming code.

Some references:
* https://arxiv.org/pdf/1601.07458.pdf
* http://www.quantum.umb.edu/Jacobs/QMT/QMT_AppendixA.pdf
* https://physics.stackexchange.com/questions/179671/how-to-take-partial-trace

To construct a reduced density matrix by taking a partial trace of a full density matrix, the dimensions of the matrix goes from $2^L \times 2^L$ to $2^n \times 2^n$. This requires a **RECTANGULAR** matrix. However, through the million definitions and tutorials and stackexchange articles I've read, none, **NONE**, explicitly stated how to construct it for the general case. They either kept it in terms of abstract notations, or they have already simplified it to a specific case. **NONE** are usable.

To add insult to injury, our system is a bit more complicated than just $ | A \rangle \otimes | B \rangle $. But rather, because we want $n$ consecutive spins in the center, the system is actually $ | B_1 \rangle \otimes | A \rangle \otimes | B_2 \rangle $!

### My current understanding / implementation of the rectangular matrix is as follows:

\begin{equation}
    \mathbb{B} = | j \rangle = | 1 \rangle_{B1} \otimes \mathbb{I}_{A} \otimes | 1 \rangle_{B2} =
    \begin{pmatrix}
        1 \\
        \vdots \\
        1
    \end{pmatrix}
    \otimes
    \begin{pmatrix}
        1 & \dots & 0 \\ 
        \vdots & \ddots  & \vdots \\
        0 & \dots & 1
    \end{pmatrix}
    \otimes
    \begin{pmatrix}
        1 \\
        \vdots \\
        1
    \end{pmatrix}
\end{equation}

Then apply this matrix to the full density matrix:

\begin{equation}
    \mathbb{B}^\dagger \mathbb{\rho} \mathbb{B}
\end{equation}

This assumes the sum over sites of $j$ in subsystem $B$ is whatever-ative, i.e.

\begin{equation}
    Tr_B(\rho) = \sum_j (\langle B_1 | \otimes \langle A | \otimes \langle B_2 |) \; \rho \; (| B_1 \rangle \otimes | A \rangle \otimes | B_2 \rangle) = (\langle 1 |_{B1} \otimes \mathbb{I}_{A} \otimes \langle 1 |_{B2}) \; \rho \; (| 1 \rangle_{B1} \otimes \mathbb{I}_{A} \otimes | 1 \rangle_{B2})
\end{equation}

# In the end, tensor contraction is used!

See function `partial_trace_tensor()`.

### Other resources

Given any orthonormal basis sets $|a_i\rangle$ and $|b_i\rangle$ for $\mathcal H_a$ and $\mathcal H_b$ respectively, any operator $K$ on the space $\mathcal H_a\otimes \mathcal H_b$ can be written:

$$K = \sum_{ij k\ell} K_{ijk\ell} |a_i\rangle |b_j\rangle \langle a_k|\langle b_\ell|$$

where $K_{ijk\ell} \equiv \langle a_i|\langle b_j| K |a_k\rangle |b_\ell\rangle$, and $\langle a_i|\langle b_j| \equiv \langle a_i|\otimes \langle b_j|$ (I omit the $\otimes$ for notational clarity).  The partial trace is then defined to be

$$\mathrm{Tr}_b(K):= \sum_{ik\ell}K_{i\ell k\ell}|a_i\rangle\langle a_k|$$

which is now a linear operator on $\mathcal H_a$ alone, with coefficients $$\bigg(\mathrm{Tr}_b(K)\bigg)_{ik} = \sum_\ell K_{i\ell k\ell}$$

Source: https://physics.stackexchange.com/questions/616061/where-does-the-expression-mathrmtrk-sum-j-1n-langle-psi-jk-psi-j/

### Test how outer product works

In [ ]:
a0 =  1 +  2j
a1 =  3 +  5j
b0 =  7 + 11j
b1 = 13 + 17j

In [ ]:
a = np.array([1 +  2j,  3 +  5j])
b = np.array([7 + 11j, 13 + 17j])
np.outer(a.conj(), b)

In [ ]:
(1 -  2j) * (7 + 11j)

In [ ]:
(1 -  2j) * (13 + 17j)

### Test bit shift (unused)

In [ ]:
num_E     = 10 # #eigenstates, ~100 - ~1000.
num_sites = 6  # #n consecutive spins.

In [ ]:
# Source: https://stackoverflow.com/questions/147713/how-do-i-manipulate-bits-in-python
def get_bit(value, n):
    return ((value >> n & 1) != 0)

def set_bit(value, n):
    return value | (1 << n)

def clear_bit(value, n):
    return value & ~(1 << n)

In [ ]:
print(get_bit(32, 4))
print(get_bit(32, 5))

In [ ]:
def drop_sites(L, sites):
    """Select sites to drop.
    This is only for demonstration. The mechanics is probably wrong."""

    idx = []
    # `i` is the index along an eigenvector.
    # `site` is the site to be dropped / summed over while constructing a reduced density matrix.
    for i in range(2**L):
        # Spins are numbered from left to right.
        # But bits are numbered from right to left.
        # Hence we need to flip the binary representation.
        for site in sites:
            print('Check site {} for removal.'.format(site))
            print('{:0{}b}'.format(i, L))
            if site == 0:
                print('^')
            else:
                print(' ' * (site) + '^')
            if get_bit(i, L - site - 1) == 1:
                idx.append(i)

    return idx

print(drop_sites(2, [0]))
print('='*25)
print(drop_sites(2, [1]))

### Code reduced density and partial trace

In [ ]:
@njit
def get_rho(V):
    """Computes a full-density matrix from an eigenvector `V`."""
    return np.outer(V, V.conj())


In [ ]:
def build_partial_trace_matrix(L, A_sites, B1_sites, B2_sites):
    """Build a rectangular matrix to take partial trace over subsystem B.
    Can only handle n consecutive sites in the middle of the system, surrounded by B1 and B2.
    |B1> x |A> x |B2>
    """

    n    = L - len(B1_sites) - len(B2_sites)
    I_A  = identity(2**len(A_sites))
    # Tr_B = csr_matrix((2**L, 2**n))

    if len(B1_sites) != 0:
        B1 = np.ones((2**len(B1_sites), 1))
        B1 = csr_matrix(B1)
    else:
        B1 = None
    if len(B2_sites) != 0:
        B2 = np.ones((2**len(B2_sites), 1))
        B2 = csr_matrix(B2)
    else:
        B2 = None

    if B1 is not None:
        Tr_B = kron(  B1, I_A, 'csr')
    else:
        Tr_B = I_A
    if B2 is not None:
        Tr_B = kron(Tr_B,  B2, 'csr')

    # if B1 is not None:
    #     print(B1.shape)
    # print(I_A.shape)
    # if B2 is not None:
    #     print(B2.shape)
    # print(Tr_B.shape)

    return Tr_B

def partial_trace_matrix(L, A_sites, B1_sites, B2_sites, V):
    """Take partial trace over subsystem B using matrix product.
    Can only handle n consecutive sites in the middle of the system, surrounded by B1 and B2.
    |B1> x |A> x |B2>
    """

    rho = get_rho(V)
    Tr_B = build_partial_trace_matrix(L, A_sites, B1_sites, B2_sites)
    rho_A = csr_matrix.dot(Tr_B.T.conj(), csr_matrix.dot(rho, Tr_B))

    return rho_A


In [ ]:
def partial_trace_tensor(L, A_sites, B1_sites, B2_sites, V):
    """Take partial trace over subsystem B using tensor contraction.
    Can only handle n consecutive sites in the middle of the system, surrounded by B1 and B2.
    |B1> x |A> x |B2>
    """

    n = len(A_sites)
    V = V.reshape([2]*L)
    rho_A = np.tensordot(V.conj(), V, axes=(B1_sites+B2_sites, B1_sites+B2_sites))
    rho_A = rho_A.reshape((2**n, 2**n))

    return rho_A


In [ ]:
def partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V):
    """Take partial trace over subsystem B using tensor contraction, but using Kevin's order of contraction.
    Can only handle n consecutive sites in the middle of the system, surrounded by B1 and B2.
    |B1> x |A> x |B2>
    """

    V = V.reshape((2**len(B1_sites), 2**len(A_sites), 2**len(B2_sites)))
    rho_A = np.tensordot(V.conj(), V, axes=([0, 2], [0, 2]))

    return rho_A


In [ ]:
def partial_trace_jonas(L, A_sites, B1_sites, B2_sites, V):
    """Take partial trace over subsystem B using tensor contraction, but using Jonas's implementation.
    Can only handle n consecutive sites in the middle of the system, surrounded by B1 and B2.
    |B1> x |A> x |B2>
    """

    dm = get_rho(V)

    # Jonas' priginal implementation:
    # dm_res = dm.reshape(2**sitesA, 2**sitesB, 2**sitesA, 2**sitesB) # rho as 4-tensor
    # rhoA = np.trace(dm_res, axis1=1, axis2=3)

    # My modification:
    dm_res = dm.reshape(2**len(B1_sites), 2**len(A_sites), 2**len(B2_sites), 2**len(B1_sites), 2**len(A_sites), 2**len(B2_sites)) # rho as 6-tensor
    dm_res = np.trace(dm_res, axis1=0, axis2=2)
    rho_A  = np.trace(dm_res, axis1=1, axis2=3)

    return rho_A


In [ ]:
Tr_B = build_partial_trace_matrix(2, [0], [1], [])
print(Tr_B.shape)
print(Tr_B.A)

In [ ]:
# Sample Hamiltonian.
L  = 8
W1 = 0.5
W2 = 8
J  = 1
periodic = False
num_Hs = 10

H1 = build_H(L, W1, J, periodic=False)
E1, V1 = ED(H1.toarray())

print('2^L: {}'.format(2**L))
print('#eigenvalues: {}'.format(len(E1)))
print('E.shape: {}'.format(E1.shape))
print('V.shape: {}'.format(V1.shape))

H2 = build_H(L, W2, J, periodic=False)
E2, V2 = ED(H2.toarray())

In [ ]:
rho = get_rho(V1[:,0])
print(rho.shape)
print(np.max(rho))
max_idx = np.unravel_index(rho.argmax(), rho.shape) # 2D index of np.argmax().
print(max_idx)

pos = max_idx[0] # Which 3x3 sub-matrix along the diagonal to print.
print(rho[pos-1:pos+2, pos-1:pos+2])

In [ ]:
# Compare matrix-generated rho_A vs tensor-generated rho_A.
A_sites  = [1,2,3,4,5,6] # Keep n consecutive
B1_sites = [0]
B2_sites = [7]
rho_A_mat1 = partial_trace_matrix(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_mat2 = partial_trace_matrix(L, A_sites, B1_sites, B2_sites, V2[:,0])
print('Theoretical size of reduced density matrix rho_A: {}'.format(2**len(A_sites)))
print('Shape of computed rho_A: {}'.format(rho_A_mat1.shape))
print('Size of computed rho_A: {}'.format(rho_A_mat1.size))

In [ ]:
rho_A_ten1 = partial_trace_tensor(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_ten2 = partial_trace_tensor(L, A_sites, B1_sites, B2_sites, V2[:,0])
print('Theoretical size of reduced density matrix rho_A: {}'.format(2**len(A_sites)))
print('Shape of computed rho_A: {}'.format(rho_A_ten1.shape))
print('Size of computed rho_A: {}'.format(rho_A_ten1.size))

In [ ]:
rho_A_kev1 = partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_kev2 = partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V2[:,0])
print('Theoretical size of reduced density matrix rho_A: {}'.format(2**len(A_sites)))
print('Shape of computed rho_A: {}'.format(rho_A_kev1.shape))
print('Size of computed rho_A: {}'.format(rho_A_kev1.size))

In [ ]:
rho_A_jon1 = partial_trace_jonas(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_jon2 = partial_trace_jonas(L, A_sites, B1_sites, B2_sites, V2[:,0])
print('Theoretical size of reduced density matrix rho_A: {}'.format(2**len(A_sites)))
print('Shape of computed rho_A: {}'.format(rho_A_jon1.shape))
print('Size of computed rho_A: {}'.format(rho_A_jon1.size))

In [ ]:
# Check Kevin's implementation and mine.
assert np.allclose(rho_A_ten1, rho_A_kev1)
assert np.allclose(rho_A_ten2, rho_A_kev2)

In [ ]:
%%timeit
rho_A_ten1 = partial_trace_tensor(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_ten2 = partial_trace_tensor(L, A_sites, B1_sites, B2_sites, V2[:,0])

In [ ]:
%%timeit
rho_A_kev1 = partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V1[:,0])
rho_A_kev2 = partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V2[:,0])
# Kevin is faster by 10-20%.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(fig_w/dpi,fig_h/dpi*2), dpi=dpi, squeeze=False)

im1 = axes[0, 0].imshow(np.abs(rho_A_mat1))
im2 = axes[0, 1].imshow(np.abs(rho_A_ten1))
im3 = axes[1, 0].imshow(np.abs(rho_A_mat2))
im4 = axes[1, 1].imshow(np.abs(rho_A_ten2))

axes[0, 0].set_title('$W=0.5 \quad \\rho_A$ computed using matrix (seems wrong)')
axes[0, 1].set_title('$W=0.5 \quad \\rho_A$ computed using tensor')
axes[1, 0].set_title('$W=8.0 \quad \\rho_A$ computed using matrix (almost correct)')
axes[1, 1].set_title('$W=8.0 \quad \\rho_A$ computed using tensor')

# plt.colorbar(im1, ax=axes[0, 0])#.set_label('Entropy')
# plt.colorbar(im2, ax=axes[0, 1])#.set_label('Entropy')

for axe in axes:
    for ax in axe:
        pass
        # ax.set_title('Both $\\rho_A$ are the same')
        # ax.legend(loc='best')
        # ax.xaxis.set_ticklabels([])
        # ax.yaxis.set_ticklabels([])
        # ax.xaxis.set_visible(False)
        # ax.yaxis.set_visible(False)

# fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(fig_w/dpi,fig_h/dpi*2), dpi=dpi, squeeze=False)

im1 = axes[0, 0].imshow(np.abs(rho_A_kev1))
im2 = axes[0, 1].imshow(np.abs(rho_A_ten1))
im3 = axes[1, 0].imshow(np.abs(rho_A_kev2))
im4 = axes[1, 1].imshow(np.abs(rho_A_ten2))

axes[0, 0].set_title('$W=0.5 \quad \\rho_A$ computed using Kevin')
axes[0, 1].set_title('$W=0.5 \quad \\rho_A$ computed using tensor')
axes[1, 0].set_title('$W=8.0 \quad \\rho_A$ computed using Kevin')
axes[1, 1].set_title('$W=8.0 \quad \\rho_A$ computed using tensor')

# plt.colorbar(im1, ax=axes[0, 0])#.set_label('Entropy')
# plt.colorbar(im2, ax=axes[0, 1])#.set_label('Entropy')

for axe in axes:
    for ax in axe:
        pass
        # ax.set_title('Both $\\rho_A$ are the same')
        # ax.legend(loc='best')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(fig_w/dpi,fig_h/dpi*2), dpi=dpi, squeeze=False)

im1 = axes[0, 0].imshow(np.abs(rho_A_jon1))
im2 = axes[0, 1].imshow(np.abs(rho_A_ten1))
im3 = axes[1, 0].imshow(np.abs(rho_A_jon2))
im4 = axes[1, 1].imshow(np.abs(rho_A_ten2))

axes[0, 0].set_title('$W=0.5 \quad \\rho_A$ computed using Jonas')
axes[0, 1].set_title('$W=0.5 \quad \\rho_A$ computed using tensor')
axes[1, 0].set_title('$W=8.0 \quad \\rho_A$ computed using Jonas')
axes[1, 1].set_title('$W=8.0 \quad \\rho_A$ computed using tensor')

# plt.colorbar(im1, ax=axes[0, 0])#.set_label('Entropy')
# plt.colorbar(im2, ax=axes[0, 1])#.set_label('Entropy')

for axe in axes:
    for ax in axe:
        pass
        # ax.set_title('Both $\\rho_A$ are the same')
        # ax.legend(loc='best')

### Batch generate data and visualize reduced density matrices for different W

In [ ]:
def batch_gen_rho_data_core(L, Ws, J=1, periodic=False, num_Hs=1000, num_EV=5, max_n=99999, clamp_zero=1e-32):

    rho_As = {}

    for W in Ws:

        Hs = build_Hs(L, W, J, periodic, num_Hs)
        E0s, V0s = EDs_sparse(Hs, num_EV)

        for E0, V0 in tqdm(zip(E0s, V0s), leave=False, desc='partial_trace()'): # Still `num_EV` eigenvectors per E0, V0.
            for i in range(len(E0)):

                A_sites  = list(range(1, L-1)) # Keep n consecutive sites, where n = L - 2.
                A_sites0 = A_sites
                parity   = 0

                while len(A_sites) != 0:

                    n = len(A_sites)

                    # Only compute reduced density matrix is size is small.
                    if n <= max_n:

                        B1_sites = list(range(0, A_sites[0]))
                        B2_sites = list(range(A_sites[-1]+1, L))
                        rho_A    = partial_trace_kevin(L, A_sites, B1_sites, B2_sites, V0[:,i])

                        # Investigate how many data points are actually zero.
                        # if n > 5:
                        #     print('W:', W)
                        #     print('rho_A <= 2'    , (np.abs(rho_A) <= 2    ).sum())
                        #     print('rho_A <= 1'    , (np.abs(rho_A) <= 1    ).sum())
                        #     print('rho_A <= 1e-8' , (np.abs(rho_A) <= 1e-8 ).sum())
                        #     print('rho_A <= 1e-12', (np.abs(rho_A) <= 1e-12).sum())
                        #     print('rho_A <= 1e-16', (np.abs(rho_A) <= 1e-16).sum())
                        #     print('rho_A <= 1e-20', (np.abs(rho_A) <= 1e-20).sum())
                        #     print('rho_A <= 1e-32', (np.abs(rho_A) <= 1e-32).sum())
                        #     print('rho_A == 0', (np.abs(rho_A) == 0).sum())
                        #     rho_A[np.abs(rho_A) <= 1e-32] = 0
                        #     print('rho_A == 0', (np.abs(rho_A) == 0).sum())

                        # If the magnitude of data is below a certain threshold, set it to zero.
                        if clamp_zero is not None:
                            rho_A[np.abs(rho_A) <= clamp_zero] = 0
                        
                        # Check if imaginary part is negligible.
                        # Conclusion: They are basically 1e-16.
                        # if n > 5:
                        #     print('W:', W)
                        #     print('Max rho_A:', np.max(np.abs(rho_A)))
                        #     print('Max real rho_A:', np.max(np.abs(np.real(rho_A))))
                        #     print('Max imag rho_A:', np.max(np.abs(np.imag(rho_A))))

                        if n not in rho_As:
                            rho_As[n] = []
                        rho_As[n].append([rho_A.astype(np.float32), W])

                    # Shorten n:
                    if parity % 3 == 0:
                        A_sites  = A_sites0[1:]
                        parity  += 1
                    elif parity % 3 == 1:
                        A_sites  = A_sites0[:-1]
                        parity  += 1
                    else:
                        A_sites  = A_sites0[1:-1]
                        A_sites0 = A_sites
                        parity   = 0
                    # print(L, len(A_sites0), len(A_sites), parity, flush=True)

    return rho_As


In [ ]:
def batch_gen_rho_data_main(L, Ws, J=1, periodic=False, num_Hs=1000, num_EV=5, max_n=99999, clamp_zero=1e-32, save_data=True):
    """Generate a list of reduced density matrices with W = {0.5, 8}, and save them to file.
    The data is used to train a classifier neutral network.

    Parameters
    ----------
    L : int
        System size.
    Ws : list of float
        A list of disorder strength W to realize.
    J : float
        Coupling strength. Always set to 1.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_Hs : int
        Number of Hamiltonians to generate, per disorder strength W.
    num_EV : int
        Number of eigenvalues around zero to use as samples.
    max_n : int
        Maximum number of consecutive sites n to store.
        Values beyond 8 will consume a GB of storage per 10000 samples.
        Recommended: 8.
    clamp_zero : float
        Threshold of reduced density matrix, magnitudes below which will be clamped to zero.

    Return
    ------
    rho_As : dict of list of list
        A `dict` of keys `n`, which are consecutive spins around the middle of the system, that the reduced density matrix is computed.
        The value of the dict is a list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. rho_As[6][i][0] is a 2D numpy.ndarray of the reduced density matrix, and rho_As[6][i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    """

    rho_As = batch_gen_rho_data_core(L, Ws, J, periodic, num_Hs, num_EV, max_n, clamp_zero)

    if save_data:
        for n, rho_A in rho_As.items():
            # For odd `n`, #samples = num_Hs * num_EV * 2
            # For even `n`, #samples = num_Hs * num_EV
            print('Writing {: 5d} `rho_A` of system size L = {:02d} of n = {:02d}.'.format(len(rho_A), L, n), flush=True)
            save_rho_train(rho_A, 'rho_A', L, n, periodic, num_EV)

    return rho_As


In [ ]:
def batch_gen_rho_data_rand(L, Ws, J=1, periodic=False, num_Hs=1000, num_EV=5, max_n=99999, clamp_zero=1e-32, save_data=True):
    """Generate a list of reduced density matrices with W != {0.5, 8}, and save them to file.
    The data is used to predict transition disorder strength W_c.

    Parameters
    ----------
    L : int
        System size.
    Ws : list of float
        A list of disorder strength W to realize.
    J : float
        Coupling strength. Always set to 1.
    periodic : bool
        Whether the Hamiltonian is periodic.
    num_Hs : int
        Number of Hamiltonians to generate, per disorder strength W.
    num_EV : int
        Number of eigenvalues around zero to use as samples.
    max_n : int
        Maximum number of consecutive sites n to store.
        Values beyond 8 will consume a GB of storage per 10000 samples.
        Recommended: 8.
    clamp_zero : float
        Threshold of reduced density matrix, magnitudes below which will be clamped to zero.

    Return
    ------
    rho_As : dict of list of list
        A `dict` of keys `n`, which are consecutive spins around the middle of the system, that the reduced density matrix is computed.
        The value of the dict is a list of lists, where each reduced density matrix is paired with its disorder strength W.
        i.e. rho_As[6][i][0] is a 2D numpy.ndarray of the reduced density matrix, and rho_As[6][i][1] is the disorder strength used to generate it.
        Number of data must be a multiple of 10.
    """

    rho_As = batch_gen_rho_data_core(L, Ws, J, periodic, num_Hs, num_EV, max_n, clamp_zero)

    if save_data:
        for n, rho_A in rho_As.items():
            # For odd `n`, #samples = num_Hs * num_EV * len(Ws) * 2
            # For even `n`, #samples = num_Hs * num_EV * len(Ws)
            print('Writing {: 5d} `rho_A` of system size L = {:02d} of n = {:02d}.'.format(len(rho_A), L, n), flush=True)
            save_rho_valid(rho_A, 'rho_A', L, n, periodic, num_EV)

    return rho_As


In [ ]:
base_samples = 25

In [ ]:
# Test drawing images of reduced density matrix.
J  = 1                      # Always = 1
Ws = [0.5, 8]               # Disorder strength W.
Ls = [8]                    # System size L.
Hs = [base_samples]         # Number of samples per L per W.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 5                  # Number of eigenvalues near zero to save.
rho_As_dict = {}

for L, num_Hs in zip(Ls, Hs):
    for p in ps:
        start_time = time.time()
        rho_As = batch_gen_rho_data_main(L, Ws, J, p, num_Hs, num_EV, max_n=10, save_data=False)
        for n, rho_A in rho_As.items():
            if n not in rho_As_dict:
                rho_As_dict[n] = []
            rho_As_dict[n] = rho_As_dict[n] + rho_A
        exec_time = time.time() - start_time
        et.append(exec_time)
        print('Computed: L={:02d} | periodic={: <5}.'.format(L, str(p)))
        print('Execution took {: 8.2f}s or {: 6.2f}min.'.format(exec_time, exec_time/60))


In [ ]:
# Test drawing images of reduced density matrix.
J  = 1                      # Always = 1
Ws = np.random.uniform(0.1, high=9.9, size=(2 * base_samples,))               # Disorder strength W.
Ls = [8]                    # System size L.
Hs = [1]                    # Number of samples per L per W.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 5                  # Number of eigenvalues near zero to save.
rho_As_rand = {}

for L, num_Hs in zip(Ls, Hs):
    for p in ps:
        start_time = time.time()
        rho_As = batch_gen_rho_data_rand(L, Ws, J, p, num_Hs, num_EV, max_n=10, save_data=False)
        for n, rho_A in rho_As.items():
            if n not in rho_As_rand:
                rho_As_rand[n] = []
            rho_As_rand[n] = rho_As_rand[n] + rho_A
        exec_time = time.time() - start_time
        et.append(exec_time)
        print('Computed: L={:02d} | periodic={: <5}.'.format(L, str(p)))
        print('Execution took {: 8.2f}s or {: 6.2f}min.'.format(exec_time, exec_time/60))


In [ ]:
n = 6
sample_idx = np.random.randint(0, len(rho_As_dict[n]), size=5*5)

fig, axes = plt.subplots(5, 5, figsize=(fig_w/dpi,fig_h/dpi*2), dpi=dpi, squeeze=False)

for i, idx in enumerate(sample_idx):
    axes[i%5,i//5].imshow(np.abs(rho_As_dict[n][idx][0]))
    axes[i%5,i//5].annotate('W={:3.1f}'.format(rho_As_dict[n][idx][1]), (0.5,0.5), xycoords='axes fraction', ha='center', color='w')

for axe in axes:
    for ax in axe:
        # ax.legend(loc='best')
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

fig.tight_layout()

In [ ]:
n = 6
sample_idx = np.random.randint(0, len(rho_As_rand[n]), size=5*5)

fig, axes = plt.subplots(5, 5, figsize=(fig_w/dpi,fig_h/dpi*2), dpi=dpi, squeeze=False)

for i, idx in enumerate(sample_idx):
    axes[i%5,i//5].imshow(np.abs(rho_As_rand[n][idx][0]))
    axes[i%5,i//5].annotate('W={:3.1f}'.format(rho_As_rand[n][idx][1]), (0.5,0.5), xycoords='axes fraction', ha='center', color='w')

for axe in axes:
    for ax in axe:
        # ax.legend(loc='best')
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

fig.tight_layout()

## Batch generate data (execution part)

In [ ]:
print('Estimated runtime to generate 10,000 samples per W:')
for i in range(7):
    print('For L = {:2d}, est runtime {:3d} min or {: 2.2f} hrs.'.format(8+i, 7 * 2**i, 7 * 2**i / 60))

In [ ]:
print('Estimated runtime to generate 100,000 samples per W:')
for i in range(7):
    print('For L = {:2d}, est runtime {:4d} min or {:5.2f} hrs.'.format(8+i, 70 * 2**i, 70 * 2**i / 60))
print(' ')
print('It will therefore be safer to break down generation to segments of 1-hr:')
for i in range(7):
    print('For L = {:2d}, separate execution into {:2d} batches with {:6d} samples each.'.format(8+i, 2**i, 100000 // 2**i))

In [ ]:
# Batch generate reduced density matrix.

k = 5
batches = 1
base_sample = 10000 // k // batches # Divide by k (num_EV)
rand_sample = 50            # Samples per random W.
Ws_main = [0.5, 8]          # Disorder strength W.
Ws_rand = np.random.uniform(0.1, high=5.9, size=(2 * base_sample // rand_sample,))
J  = 1                      # Always = 1
Ls = list(range(8,15,2))      # System sizes L.
ps = [False, True]          # Periodic or not.
et = []                     # Execution time.
Hs_main = [base_sample]*len(Ls) # Number of samples per L per W.
Hs_rand = [rand_sample]*len(Ls) # Number of samples per L per W.
num_EVs = [k]               # Number of eigenvalues near zero to save.

for i in range(batches):
    print('{} | Processing batch {:03d} of {:d}:'.format(dt(), i+1, batches), flush=True)
    print(' ', flush=True)
    for L, num_Hs_m, num_Hs_r in zip(Ls, Hs_main, Hs_rand):
        for num_EV in num_EVs:
            for p in ps:
                start_time = time.time()
                print('{} | Generating training data for L={:02d}...'.format(dt(), L), flush=True)
                batch_gen_rho_data_main(L, Ws_main, J, p, num_Hs_m, num_EV, max_n=6, clamp_zero=1e-32, save_data=True)
                print('{} | Generating random data for L={:02d}...'.format(dt(), L), flush=True)
                batch_gen_rho_data_rand(L, Ws_rand, J, p, num_Hs_r, num_EV, max_n=6, clamp_zero=1e-32, save_data=True)
                exec_time = time.time() - start_time
                et.append(exec_time)
                print('{} | Computed: L={:02d} | num_EV={} | periodic={: <5}.'.format(dt(), L, num_EV, str(p)), flush=True)
                print('{} | Execution took {: 8.2f}s or {: 6.2f}min.'.format(dt(), exec_time, exec_time/60), flush=True)
                print(' ', flush=True)

    if check_shutdown_signal():
        break

## Neural network
See the other notebook for training and prediction.

In [ ]:
data = load_rho_train('rho_A', L=8, n=4, periodic=False, num_EV=1)

In [ ]:
type(data)

In [ ]:
len(data) # [[rho_A, W], [rho_A, W], [rho_A, W], ...]

In [ ]:
type(data[0])

In [ ]:
len(data[0]) # [rho_A, W]